In [1]:
import os
import torch

import numpy as np

from liger import Liger, Flyingsquid_Cluster

from core import load_config
from utils import evaluate_thresholds, cluster_embeddings

/hdd2/dyah/anaconda3/envs/liger/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cfg = load_config('configs/spam.yaml')
n_clusters = cfg['n_clusters']
engine = "ada"
dataset = cfg['dataset']
thresholds = cfg['thresholds']
T = cfg['T']
embedding_path = cfg['embedding_path']
data_path = embedding_path

Let's load L, Y matrix, and the embeddings

In [3]:
L_dev_raw_orig = torch.load(os.path.join(data_path, 'val_L.pt')).detach().cpu().numpy()
Y_dev_raw = torch.load(os.path.join(data_path, 'val_Y.pt')).detach().cpu().numpy()
Y_test_raw = torch.load(os.path.join(data_path, 'test_Y.pt')).detach().cpu().numpy()
L_train_raw_orig = torch.load(os.path.join(data_path, 'train_L.pt')).detach().cpu().numpy()
L_test_raw_orig = torch.load(os.path.join(data_path, 'test_L.pt')).detach().cpu().numpy()

avg_embeddings_train = torch.load(os.path.join(embedding_path, 'train_feature.pt')).detach().cpu().numpy()
avg_embeddings_dev = torch.load(os.path.join(embedding_path,'val_feature.pt')).detach().cpu().numpy()
avg_embeddings_test = torch.load(os.path.join(embedding_path,'test_feature.pt')).detach().cpu().numpy()

## Baseline: FlyingSquid without label expansion

In [4]:
neg_balances_to_try = np.arange(.01, .99, .05)
tune_by = 'acc'
T = 1

In [5]:
L_train = L_train_raw_orig[:L_train_raw_orig.shape[0] - (L_train_raw_orig.shape[0] % T)]
L_dev = L_dev_raw_orig[:L_dev_raw_orig.shape[0] - (L_dev_raw_orig.shape[0] % T)]
L_test = L_test_raw_orig[:L_test_raw_orig.shape[0] - (L_test_raw_orig.shape[0] % T)]
Y_dev = Y_dev_raw[:Y_dev_raw.shape[0] - (Y_dev_raw.shape[0] % T)]
Y_test = Y_test_raw[:Y_test_raw.shape[0] - (Y_test_raw.shape[0] % T)]

thresholds = [1 for i in range(L_train.shape[1])]

m_per_task = L_train.shape[1]
m = T * m_per_task
v = T

kmeans, embedding_groups, train_cluster_labels = cluster_embeddings(avg_embeddings_train, n_clusters)
dev_cluster_labels = kmeans.predict(avg_embeddings_dev)
test_cluster_labels = kmeans.predict(avg_embeddings_test) 

cluster_models = []
for i in range(len(embedding_groups)):
    cluster_models.append(Flyingsquid_Cluster(X=embedding_groups[i], mu=kmeans.cluster_centers_[i], T=T, m_per_task=m_per_task))
    
outputs = evaluate_thresholds(thresholds, cluster_models, neg_balances_to_try, \
    L_train, L_dev, L_test, \
    Y_dev, Y_test, train_cluster_labels, dev_cluster_labels, test_cluster_labels,\
    evaluate_test=False, tune_test=True, tune_by=tune_by)

/hdd2/dyah/anaconda3/envs/liger/lib/python3.10/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(


Dev Thresholds: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Dev Acc: 85.00%	Pre: 95.35%	Rec: 75.93%	F1: 84.54%
Test Thresholds: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Test Acc: 85.60%	Pre: 86.61%	Rec: 82.20%	F1: 84.35%


## LIGER: Expand labels via embeddings

In [6]:
thresholds = cfg['thresholds']

liger = Liger()
L_train_expanded = liger.expand_lfs(
    L_train_raw_orig, L_train_raw_orig, avg_embeddings_train, avg_embeddings_train,
    thresholds = thresholds)
L_dev_expanded = liger.expand_lfs(
    L_train_raw_orig, L_dev_raw_orig, avg_embeddings_train, avg_embeddings_dev,
    thresholds = thresholds)
L_test_expanded = liger.expand_lfs(
    L_train_raw_orig, L_test_raw_orig, avg_embeddings_train, avg_embeddings_test,
    thresholds = thresholds)

L_train = L_train_expanded[:L_train_expanded.shape[0] - (L_train_expanded.shape[0] % T)]
L_dev = L_dev_expanded[:L_dev_expanded.shape[0] - (L_dev_expanded.shape[0] % T)]
L_test = L_test_expanded[:L_test_expanded.shape[0] - (L_test_expanded.shape[0] % T)]
Y_dev = Y_dev_raw[:Y_dev_raw.shape[0] - (Y_dev_raw.shape[0] % T)]
Y_test = Y_test_raw[:Y_test_raw.shape[0] - (Y_test_raw.shape[0] % T)]

m_per_task = L_train.shape[1]

m = T * m_per_task
v = T

kmeans, embedding_groups, train_cluster_labels = cluster_embeddings(avg_embeddings_train, n_clusters)
dev_cluster_labels = kmeans.predict(avg_embeddings_dev)
test_cluster_labels = kmeans.predict(avg_embeddings_test) 
cluster_models = []

for i in range(len(embedding_groups)):
    cluster_models.append(Flyingsquid_Cluster(X=embedding_groups[i], mu=kmeans.cluster_centers_[i], T=T, m_per_task=m_per_task))
    
outputs = evaluate_thresholds(thresholds, cluster_models, neg_balances_to_try, \
    L_train_expanded, L_dev_expanded, L_test_expanded, \
    Y_dev_raw, Y_test_raw, train_cluster_labels, dev_cluster_labels, test_cluster_labels,\
    evaluate_test=False, tune_test=True, tune_by=tune_by)

/hdd2/dyah/anaconda3/envs/liger/lib/python3.10/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(


Dev Thresholds: [0.844, 0.864, 0.854, 0.844, 0.844, 0.844, 0.844, 0.804, 0.844, 0.844]
Dev Acc: 96.00%	Pre: 98.08%	Rec: 94.44%	F1: 96.23%
Test Thresholds: [0.844, 0.864, 0.854, 0.844, 0.844, 0.844, 0.844, 0.804, 0.844, 0.844]
Test Acc: 93.60%	Pre: 93.97%	Rec: 92.37%	F1: 93.16%
